In [ ]:
# pip install --upgrade threadpoolctl

### Import packages

In [ ]:
import pandas as pd, numpy as np
from matplotlib import pyplot as plt

### Graphical parameters

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15, 6) # (w, h)

### Set path for dataset

In [ ]:
path = # insert path to data here

### Import dataset into pandas dataframe and sort by stock + date

In [ ]:
df = pd.read_csv(path + "stocks.csv")
df = df.sort_values(by=['PERMNO', "date"])
print(df.shape)
df.head()

### Table of each firm's activity sector

In [ ]:
sectors = # 1 row per stock with columns |TICKER|sector_1dgt|
sectors.head()

### Rearrange data to get dates in first column and one stock per column

In [ ]:
df1 = # insert code here
print(df1.shape)
df1.head()

### Build train and test set

In [ ]:
train = # first 134 observations of stocks 
test = # next observations until end of dataframe
train_dates = train.index.to_list()

### Compute each stock's cumulated return for graphical display

In [ ]:
cum_ret = train.cumsum()
cum_ret.head()

In [ ]:
cum_ret.plot(legend=False) # legend=False to disable legend display due to the high number of stocks (300)

### Rearrange data with stocks in rows and returns in columns to treat returns as stock features

In [ ]:
data_pivot = # insert code here
print(data_pivot.shape)
data_pivot.head()

### Compute 8 clusters based on return behavior on the train period

In [ ]:
from sklearn.cluster import KMeans
kmeans = # intantiate KMeans with 8 clusters, random_state=0. Then train model (fit method)
clusters = pd.DataFrame(kmeans.cluster_centers_)
clusters

### <u>Question</u>: give the interpretation of each row in the *clusters* table

### Transpose table so as to get centroid returns in colums and plot cumulated returns by cluster

In [ ]:
clust_data = # transposed "clusters" table
nb_clusters = # retrieve number of clusters
New_labels=["Cluster " + str(x) for x in range(nb_clusters)] # Create cluster names
clust_data.columns = New_labels
clust_data.head()

In [ ]:
cum_ret = clust_data.cumsum() # compute cumulated returns
dates = data_pivot.columns # retrieve dates. Objective : create plot with dates on horizontal axis
cum_ret["date"] = dates # add date column in cum_ret
cum_ret = cum_ret.set_index(["date"]) # set "date" column as the index (will be considered as x values in plot)
cum_ret.plot()

### Retrieve the cluster a firm belongs to

In [ ]:
clust = # use kmeans object predict method
clust

### Count the number of firms in each cluster

In [ ]:
data_temp = data_pivot.copy() # we want to leave data_pivot untouched
data_temp["cluster"] = # assign to each firm the cluster it belongs to based on 'clust' array
# count number of firms per cluster using 'groupby'

### Do clusters match activity sector of firms?

In [ ]:
data_temp2 = pd.DataFrame(data_temp["cluster"]).reset_index() # keep cluster values only
match_sector = # merge data_temp2 with sectors table
# report nobs, mean, min, max and std of sector for each cluster

### <u>Question</u>: do clusters match the activity sector of firms?

### Returns of equally-weighted portfolio from each cluster

In [ ]:
PF = # compute time series of returns of each cluster portfolio
PF_cumret = # compute cumulated returns of each portfolio
PF_cumret.plot()

### <u>Question</u>: compare with centroid cumulated returns graph. Why do we get the same graphs?

### How many clusters?

In [ ]:
error = []
for i in np.arange(1, 50):
    kmeans = # train model with i clusters. Set random_state to 0 to get comparable results
    #append computed error to error list
errors = pd.DataFrame(error)

In [ ]:
plt.plot(np.arange(1, 50), error, "b-")

Elbow rule: To determine the optimal number of clusters, we have to select the value of k at the “elbow” i.e. the point after which the inertia starts decreasing in a linear fashion. No clear k here... 10 seems to be a reasonable choice however.

In [ ]:
kmeans_optim = KMeans(n_clusters=10, random_state=0).fit(data_pivot)
clusters = pd.DataFrame(kmeans_optim.cluster_centers_)

clust_data = clusters.T
nb_clusters = clusters.shape[0] # retrieve number of clusters
New_labels=["Cluster " + str(x) for x in range(nb_clusters)] # Build cluster names
clust_data.columns = New_labels
clust_data.head()

In [ ]:
cum_ret = clust_data.cumsum() # compute cumulated returns
dates = data_pivot.columns # retrieve dates. Objective : create plot with dates on horizontal axis
cum_ret["date"] = dates # create date column in cum_ret
cum_ret = cum_ret.set_index(["date"]) # set "date" column as the index (will be considered as x values in plots)
cum_ret.plot()

### Number of stocks in each cluster

In [ ]:
data_temp = data_pivot.copy() # we want to leave data_pivot untouched
data_temp["cluster"] = kmeans_optim.predict(data_pivot)
nb_stocks = data_temp.groupby("cluster")["cluster"].count()
nb_stocks

### Select the most representative stock within each cluster, i.e. the one closest to the cluster centroid

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
closest, _ = pairwise_distances_argmin_min(kmeans_optim.cluster_centers_, data_pivot)
closest

In [ ]:
best_fits = # for each cluster, select the stock that is closest to the cluster centroid
cum_best_fits = # compute cumulated returns of each selected stock
cum_best_fits.head()
cum_best_fits.round(decimals=3).head()

### <u>Question</u>: there are 2 clusters with one stock only. Which stocks are in thse clusters? Does it make sense that these stocks are alone in their cluster?

### Plot of selected stock returns and centroid returns

In [ ]:
fig, axs = plt.subplots(5, 2)
fig.tight_layout(pad=3.0)
cum_best_fits_cols = cum_best_fits.columns
cum_ret_cols = cum_ret.columns
for i in range(len(cum_best_fits_cols)):
    row = int(i/2)
    j = i%2
    cur_col_best_fits = cum_best_fits_cols[i]
    cur_col_cum_ret = cum_ret_cols[i]
    s = pd.concat([cum_best_fits[cur_col_best_fits],
                   cum_ret[cur_col_cum_ret]], axis=1).reset_index(drop=True) # reset_index + drop otherwise dates
                                                                             # are reported on x axis and are unreadable
    axs[row,j].plot(s)
    axs[row,j].title.set_text("Nb stocks = {}".format(nb_stocks[i]))

### Plot of selected stock returns against centroid returns

In [ ]:
best_fits2 = best_fits.T
best_fits2 = best_fits2.reset_index(drop=True)

fig, axs = plt.subplots(5, 2)
fig.tight_layout(pad=3.0)
best_fits2_cols = best_fits2.columns
ret_cols = clust_data.columns
for i in range(len(best_fits2_cols)):
    row = int(i/2)
    j = i%2
    cur_col_best_fits2 = best_fits2_cols[i]
    cur_col_ret = ret_cols[i]
    s1 = best_fits2[cur_col_best_fits2]
    s2 = clust_data[cur_col_ret]
    axs[row,j].plot(s1,s2, "ro", ms=1)
    axs[row,j].axline([0, 0], [1, 1])
    axs[row,j].title.set_text("Nb stocks = {}".format(nb_stocks[i]))
    

### Can we replicate the whole portfolio (300 stocks) with the 10 representative stocks only?

### Performance on train set

#### Correlation bewteen whole portfolio returns and 10-stock portfolio returns

In [ ]:
ret_PF_all = # compute the time series of returns of the 300-stock equally-weighted porfolio on train set
ret_PF_repr_stocks =  # compute the time series of returns of the 10-stock equally-weighted porfolio on train set
# compute the correlation in returns of the two portfolios

### <u>Question</u>: what is the quality of the replication of the 300-stock portfolio by the 10-stock portfolio on the train set?

#### Plot of returns

In [ ]:
ret_all_train = pd.DataFrame(ret_PF_all)
ret_repr_train = pd.DataFrame(ret_PF_repr_stocks)
ret_train = pd.concat([ret_all_train, ret_repr_train], axis=1)
ret_train.columns = ['all', 'selected']
ret_train.plot()

### Performance on test set

#### Returns

In [ ]:
ret_PF_all = # compute the time series of returns of the 300-stock equally-weighted porfolio on train set
best_fits = # retrieve the returns of the 10 stocks on the test set
ret_PF_repr_stocks = # compute the time series of returns of the 10-stock equally-weighted porfolio on test set
# compute the correlation in returns of the two portfolios

### <u>Question</u>: what is the quality of the replication of the 300-stock portfolio by the 10-stock portfolio on the test set?

#### Plot of returns

In [ ]:
ret_all_test = pd.DataFrame(ret_PF_all)
ret_repr_test = pd.DataFrame(ret_PF_repr_stocks)
ret_test = pd.concat([ret_all_test, ret_repr_test], axis=1)
ret_test.columns = ['all', 'selected']
ret_test.plot()